# Result Analysis

This notebook calculate multiple structural feature of predicted AlphaFold structures.

- Hydrogen bonds
- SASA

In [1]:
# Requirements and dependencies, defined some functions for PyMOL analysis
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import json
from pymol import cmd, CmdException
import os,time
from rdkit import Chem

# import some structure analysis functions
from structure_analysis import *


# all possible pdb names
model_names = ["relaxed_model_%d_ptm_pred_0"%i for i in range(1,6)] + ["unrelaxed_model_%d_ptm_pred_0"%i for i in range(1,6)]
unrelaxed_model_names = ["unrelaxed_model_%d_ptm_pred_0"%i for i in range(1,6)]

In [2]:
# create dataframe object for A501 proteome
df_A501 = pd.DataFrame(columns=["ID","name","sequence"])

# if you used the full proteome, you can use the following code to load the dataframe
# df_A501["ID"] = ["A501_%d"%(i+1) for i in range(302+1878)]   # uncomment this line to use the full proteome
# here we use the short proteome, you can use the following code to load the dataframe
df_A501["ID"] = ["A501_%d"%(i+1) for i in range(10)]
df_A501.head()

,ID,name,sequence
0,A501_1,NaN,NaN
1,A501_2,NaN,NaN
2,A501_3,NaN,NaN
3,A501_4,NaN,NaN
4,A501_5,NaN,NaN


In [3]:
# Feature in this step: sequence, structure available, sequence length
structure_avail_list = []
sequence_list = []
sequence_len_list = []

for filename in df_A501["ID"]:
    structure_avail = min([os.path.exists("A501_structure/%s/%s.pdb"%(filename,model_name)) for model_name in model_names])
    if structure_avail:
        structure_avail_list.append(True)    # if all 5 models are available, then True
        cmd.load("A501_structure/%s/relaxed_model_1_ptm_pred_0.pdb"%filename,"A501")  # load the structure
        sequence = ''.join(cmd.get_fastastr("all").split('\n')[1:])     # get sequence
        sequence_list.append(sequence)
        sequence_len_list.append(len(sequence))
        cmd.reinitialize()
    else:
        structure_avail_list.append(False)
        sequence_list.append(None)
        sequence_len_list.append(None)


# add the sequence and structure availability to the dataframe
df_A501["sequence"] = sequence_list
df_A501["sequence_length"] = sequence_len_list
df_A501["structure_avail"] = structure_avail_list
df_A501.head()

 PyMOL not running, entering library mode (experimental)


,ID,name,sequence,sequence_length,structure_avail
0,A501_1,NaN,MAKKGAGATRGISPVRPTRALPIGAYLKVADNSGAKVIQIIGVVGY...,141,True
1,A501_2,NaN,MGKSLIQQRRGKGTTTFRAPSHRYRGAVKYVPLNVVKEKTLRGVVE...,239,True
2,A501_3,NaN,MKVVRFGVSVPEELLEKFDRIIEEKGYVNRSEAIRDLMRDFIIRHE...,138,True
3,A501_4,NaN,MDCTKDYCVKDLSLATSGEKKIDWVSRFMPVLQTIRREFEREKPFK...,421,True
4,A501_5,NaN,MRRDYTLYLFASLGSFLIAYIALPLVIIFAKQLMDWEMLVKTLHDP...,247,True


In [4]:
# Feature in this step: plddt for all 5 unrelaxed models (relaxed models' plddt is equal to unrelaxed models)
template_list_plddt = []
for index in df_A501["ID"]:
    if os.path.exists("A501_structure/%s/ranking_debug.json"%index):
        f_json = open("A501_structure/%s/ranking_debug.json"%index,"r")
        json_dict = json.load(f_json)
        
        # read plddt from json file
        plddt_1 = json_dict["plddts"]["model_1_ptm_pred_0"]
        plddt_2 = json_dict["plddts"]["model_2_ptm_pred_0"]
        plddt_3 = json_dict["plddts"]["model_3_ptm_pred_0"]
        plddt_4 = json_dict["plddts"]["model_4_ptm_pred_0"]
        plddt_5 = json_dict["plddts"]["model_5_ptm_pred_0"]
        
        template_list_plddt.append([index,plddt_1,plddt_2,plddt_3,plddt_4,plddt_5])

    else:
        template_list_plddt.append([index,None,None,None,None,None])

# create a dataframe for plddt
template_df_plddt = pd.DataFrame(template_list_plddt,columns=["ID","model_1_ptm_plddt","model_2_ptm_plddt","model_3_ptm_plddt","model_4_ptm_plddt","model_5_ptm_plddt"])
df_A501 = df_A501.merge(template_df_plddt,how="left",on="ID")
df_A501.head()

,ID,name,sequence,sequence_length,structure_avail,model_1_ptm_plddt,model_2_ptm_plddt,model_3_ptm_plddt,model_4_ptm_plddt,model_5_ptm_plddt
0,A501_1,NaN,MAKKGAGATRGISPVRPTRALPIGAYLKVADNSGAKVIQIIGVVGY...,141,True,89.662483,91.420092,91.683542,90.894946,92.880347
1,A501_2,NaN,MGKSLIQQRRGKGTTTFRAPSHRYRGAVKYVPLNVVKEKTLRGVVE...,239,True,93.713796,93.865745,90.135302,90.741828,87.179584
2,A501_3,NaN,MKVVRFGVSVPEELLEKFDRIIEEKGYVNRSEAIRDLMRDFIIRHE...,138,True,95.309920,94.801728,90.436254,91.448782,90.237134
3,A501_4,NaN,MDCTKDYCVKDLSLATSGEKKIDWVSRFMPVLQTIRREFEREKPFK...,421,True,96.689124,96.717610,96.477178,96.680157,96.080827
4,A501_5,NaN,MRRDYTLYLFASLGSFLIAYIALPLVIIFAKQLMDWEMLVKTLHDP...,247,True,95.212583,95.177905,94.975812,93.970114,94.203397


In [5]:
# Feature in this step: number of disulfide bonds, hydrogen bonds, salt bridges, relative surface area, secondary structure
# Feature in this step: chirality of Ca, phi, psi, omega
# Calculate protein properties, this may take a while
disu_overall_list = []    # disulfide bonds
hbon_overall_list = []    # hydrogen bonds
sbrg_overall_list = []    # salt bridges
surf_overall_list = []    # relative surface area
sasa_overall_list = []    # solvent accessible surface area
sstr_overall_list = []    # secondary structure
chir_overall_list = []    # chirality, central carbon
phiv_overall_list = []    # phi angle
psiv_overall_list = []    # psi angle
omgv_overall_list = []    # omega angle, current residue to next residue

for i in range(df_A501.shape[0]):
    disu_list = []
    hbon_list = []
    sbrg_list = []
    surf_list = []
    sasa_list = []
    sstr_list = []
    chir_list = []
    phiv_list = []
    psiv_list = []
    omgv_list = []
    if df_A501["structure_avail"][i]:
        for model_name in model_names:
            cmd.load("A501_structure/%s/%s.pdb"%(df_A501["ID"][i],model_name),"%s"%df_A501["ID"][i])   # load pdb file
    
            cmd.distance("disul","name SG","name SG",cutoff=2.1)     # disulfide bond
            cmd.distance("hbond","all","all",mode=2)                 # hydrogen bond
            cmd.distance("saltbridge","(resn ASP and name OD2) or (resn GLU and name OD2)","(resn ARG and (name NH1 or name NH2)) or (resn LYS and name NZ) or (resn HIS and name NE2)",cutoff=4)  # salt bridge
            cmd.dss()           # secondary structure
            cmd.assign_stereo() # assign chirality
            ss_string = ""
            for a in cmd.get_model(df_A501["ID"][i] +" and n. ca").atom:
                ss_string = ss_string+a.ss 
            
            chir_protein_list = []
            phiv_protein_list = []
            psiv_protein_list = []
            omgv_protein_list = []

            chirality_space = {'chir_protein_list': chir_protein_list}
            cmd.iterate('name CA', 'chir_protein_list.append(stereo)', space=chirality_space)

            for res_num in range(1,df_A501["sequence_length"][i]+1):
                # first residue has no phi
                if res_num > 1:
                    phi = cmd.get_dihedral("%d/c"%(res_num-1), "%d/n"%res_num , "%d/ca"%res_num   , "%d/c"%res_num     )
                    phiv_protein_list.append(phi)

                # last residue has no psi and omega
                if res_num < df_A501["sequence_length"][i]:
                    psi = cmd.get_dihedral("%d/n"%res_num    , "%d/ca"%res_num, "%d/c"%res_num    , "%d/n"%(res_num+1) )
                    psiv_protein_list.append(psi)
                    omg = cmd.get_dihedral("%d/ca"%res_num   , "%d/c"%res_num , "%d/n"%(res_num+1), "%d/ca"%(res_num+1))
                    omgv_protein_list.append(omg)

    
            disu_list.append(len(get_raw_distances("disul")))       # disulfide bond
            hbon_list.append(len(get_raw_distances("hbond")))       # hydrogen bond
            sbrg_list.append(len(get_raw_distances("saltbridge")))  # salt bridge
            surf_list.append(cmd.get_area('all'))                   # protein surface
            cmd.set('dot_solvent',1)
            sasa_list.append(cmd.get_area('all'))                   # SASA
            sstr_list.append(ss_string)                             # secondary structure
            chir_list.append("".join(chir_protein_list))                      # chirality
            phiv_list.append(phiv_protein_list)                     # phi angle
            psiv_list.append(psiv_protein_list)                     # psi angle
            omgv_list.append(omgv_protein_list)                     # omega angle
    
            cmd.reinitialize()
    else:
        disu_list.append(None)       # disulfide bond
        hbon_list.append(None)       # hydrogen bond
        sbrg_list.append(None)       # salt bridge
        surf_list.append(None)       # protein surface
        sasa_list.append(None)       # SASA
        sstr_list.append(None)       # secondary structure
        chir_list.append(None)       # chirality
        phiv_list.append(None)       # phi angle
        psiv_list.append(None)       # psi angle
        omgv_list.append(None)       # omega angle

    
    # add to the list
    disu_overall_list.append(disu_list)
    hbon_overall_list.append(hbon_list)
    sbrg_overall_list.append(sbrg_list)
    surf_overall_list.append(surf_list)
    sasa_overall_list.append(sasa_list)
    sstr_overall_list.append(sstr_list)
    chir_overall_list.append(chir_list)
    phiv_overall_list.append(phiv_list)
    psiv_overall_list.append(psiv_list)
    omgv_overall_list.append(omgv_list)


    print("\r%d/%d"%(i+1,df_A501.shape[0]),end="")


 RDKit-Warning: Explicit valence for atom # 0 N, 4, is greater than permitted
 Using method=rdkit
 RDKit-Warning: Explicit valence for atom # 0 N, 4, is greater than permitted
 Using method=rdkit
 RDKit-Warning: Explicit valence for atom # 0 N, 4, is greater than permitted
 Using method=rdkit
 RDKit-Warning: Explicit valence for atom # 0 N, 4, is greater than permitted
 Using method=rdkit
 RDKit-Warning: Explicit valence for atom # 0 N, 4, is greater than permitted
 Using method=rdkit
 RDKit-Warning: Explicit valence for atom # 38 O, 3, is greater than permitted
 Using method=rdkit
 RDKit-Warning: Explicit valence for atom # 38 O, 3, is greater than permitted
 Using method=rdkit
 RDKit-Warning: Explicit valence for atom # 25 O, 3, is greater than permitted
 Using method=rdkit
 RDKit-Warning: Explicit valence for atom # 16 O, 3, is greater than permitted
 Using method=rdkit
 RDKit-Warning: Explicit valence for atom # 25 O, 3, is greater than permitted
 Using method=rdkit
1/10 RDKit-Warn

In [6]:
# Add calculated data to dataframe
df_A501["disulfide_bond"] = disu_overall_list
df_A501["hydrogen_bond"] = hbon_overall_list
df_A501["salt_bridge"] = sbrg_overall_list
df_A501["surface"] = surf_overall_list
df_A501["SASA"] = sasa_overall_list
df_A501["secondary_structure"] = sstr_overall_list
df_A501["chirality"] = chir_overall_list
df_A501["phi_angle"] = phiv_overall_list
df_A501["psi_angle"] = psiv_overall_list
df_A501["omega_angle"] = omgv_overall_list



df_A501.to_csv("A501_results.csv",index=False)  # save A501 result to csv file
df_A501.head()

,ID,name,sequence,sequence_length,structure_avail,model_1_ptm_plddt,model_2_ptm_plddt,model_3_ptm_plddt,model_4_ptm_plddt,model_5_ptm_plddt,disulfide_bond,hydrogen_bond,salt_bridge,surface,SASA,secondary_structure,chirality,phi_angle,psi_angle,omega_angle
0,A501_1,NaN,MAKKGAGATRGISPVRPTRALPIGAYLKVADNSGAKVIQIIGVVGY...,141,True,89.662483,91.420092,91.683542,90.894946,92.880347,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]","[120, 124, 121, 120, 117, 110, 122, 122, 115, ...","[2, 3, 3, 3, 3, 1, 1, 1, 2, 2]","[16453.87890625, 16443.955078125, 16382.051757...","[8940.4306640625, 8955.91796875, 8809.41992187...",[LLLLLLLLLLLLLLLLLLLLLLLSSSSSSLLLLLLSSSSSSSSLL...,[SSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSS...,"[[-83.25824737548828, -53.65321731567383, -60....","[[92.93165588378906, 113.14421844482422, 111.6...","[[176.8803253173828, 169.42523193359375, 162.6..."
1,A501_2,NaN,MGKSLIQQRRGKGTTTFRAPSHRYRGAVKYVPLNVVKEKTLRGVVE...,239,True,93.713796,93.865745,90.135302,90.741828,87.179584,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]","[178, 179, 176, 176, 173, 173, 170, 165, 174, ...","[1, 1, 3, 2, 1, 0, 0, 2, 1, 0]","[28244.298828125, 28293.125, 28136.4375, 28152...","[15989.1181640625, 16348.2314453125, 16080.538...",[LLLLLHHHHHHHLLLLLLLLHHHLLLLLLLLLHHHHLLLLSSSSS...,[SSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSS...,"[[-98.37919616699219, -82.6944580078125, -58.7...","[[155.01051330566406, 172.655517578125, 164.13...","[[-174.631103515625, 170.70396423339844, 171.4..."
2,A501_3,NaN,MKVVRFGVSVPEELLEKFDRIIEEKGYVNRSEAIRDLMRDFIIRHE...,138,True,95.309920,94.801728,90.436254,91.448782,90.237134,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]","[144, 150, 146, 149, 148, 136, 144, 138, 139, ...","[6, 5, 2, 6, 2, 5, 3, 2, 4, 1]","[16813.251953125, 16833.84765625, 16781.255859...","[10152.1630859375, 10112.7919921875, 10369.317...",[LLLLLLLLLLLHHHHHHHHHHHHHHLLLLHHHHHHHHHHHHHHHH...,[SSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSS...,"[[-91.49331665039062, -84.85379791259766, -112...","[[135.04815673828125, 122.14190673828125, 133....","[[-175.10952758789062, 172.95986938476562, 177..."
3,A501_4,NaN,MDCTKDYCVKDLSLATSGEKKIDWVSRFMPVLQTIRREFEREKPFK...,421,True,96.689124,96.717610,96.477178,96.680157,96.080827,"[1, 1, 1, 1, 1, 0, 0, 0, 0, 0]","[520, 529, 532, 530, 534, 501, 507, 518, 511, ...","[17, 16, 12, 14, 12, 12, 13, 9, 11, 11]","[49280.71875, 49317.3046875, 49249.125, 49186....","[19155.99609375, 19129.640625, 19248.982421875...",[LSSSLLSSSSLHHHHHHHHHHHHHHHHHLHHHHHHHHHHHHHLLL...,[SSRSSSSRSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSS...,"[[-100.90159606933594, -122.80050659179688, -1...","[[122.10027313232422, 106.00759887695312, 131....","[[174.44021606445312, -176.1517791748047, 179...."
4,A501_5,NaN,MRRDYTLYLFASLGSFLIAYIALPLVIIFAKQLMDWEMLVKTLHDP...,247,True,95.212583,95.177905,94.975812,93.970114,94.203397,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]","[311, 316, 308, 317, 312, 321, 335, 327, 328, ...","[3, 2, 2, 2, 3, 2, 1, 2, 1, 2]","[29042.462890625, 29122.55859375, 29035.378906...","[15279.736328125, 15412.810546875, 15343.02441...",[LLLLHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHLHHHHHHHHHL...,[SSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSS...,"[[-67.35725402832031, -57.368064880371094, -78...","[[103.6756362915039, 115.55360412597656, 121.9...","[[168.47445678710938, 171.64271545410156, -178..."


In [ ]:
# start a similar process for 3DAC proteome
df_3DAC = pd.DataFrame(columns=["ID","name","sequence"])

# if you used the full proteome, you can use the following code to load the dataframe
# df_3DAC["ID"] = ["3DAC_%d"%(i+1) for i in range(302+1196)]   # uncomment this line to use the full proteome
# here we use the short proteome, you can use the following code to load the dataframe
df_3DAC["ID"] = ["3DAC_%d"%(i+1) for i in range(10)]
df_3DAC.head()

,ID,name,sequence
0,3DAC_1,NaN,NaN
1,3DAC_2,NaN,NaN
2,3DAC_3,NaN,NaN
3,3DAC_4,NaN,NaN
4,3DAC_5,NaN,NaN


In [ ]:
structure_avail_list = []
sequence_list = []
sequence_len_list = []
for filename in df_3DAC["ID"]:
    try:
        os.rename("3DAC_structure/%s/unrelaxed_model_1_ptm.pdb"%filename,"3DAC_structure/%s/unrelaxed_model_1.pdb"%filename)
        os.rename("3DAC_structure/%s/unrelaxed_model_2_ptm.pdb"%filename,"3DAC_structure/%s/unrelaxed_model_2.pdb"%filename)
    except:
        pass
    if os.path.exists("3DAC_structure/%s/unrelaxed_model_1.pdb"%filename):
        structure_avail_list.append(True)
        cmd.load("3DAC_structure/%s/unrelaxed_model_1.pdb"%filename,"3DAC")
        sequence = ''.join(cmd.get_fastastr("all").split('\n')[1:])
        sequence_list.append(sequence)
        sequence_len_list.append(len(sequence))
        cmd.reinitialize()
    else:
        structure_avail_list.append(False)
        sequence_list.append(None)
        sequence_len_list.append(None)
df_3DAC["sequence"] = sequence_list
df_3DAC["sequence length"] = sequence_len_list
df_3DAC["structure avail"] = structure_avail_list

df_3DAC.head()

,ID,name,sequence,sequence length,structure avail
0,3DAC_1,NaN,MAQKKGSSQKLLVWILVGFALGIVGGLILGKDNVIWVAWMGDVFIR...,409,True
1,3DAC_2,NaN,MSTMPSKVVGIIGGMGPEAGVDLVYKVIKLSPAKRDQEHIHVILDN...,232,True
2,3DAC_3,NaN,MAQVKEVRLYTYQWKAEPPIGVSVLIIHGLGEHAGRYKSLVKALND...,264,True
3,3DAC_4,NaN,MYSQFKKLVKDAIRKADIEKRLDEIITYRRELHQIPEVGLELPKTK...,394,True
4,3DAC_5,NaN,MNEMFVVQAKFIGKESQKKLEGKTVAIVGCGGLGSPLVQMAARSGI...,230,True


In [ ]:
# Calculate plddt
template_list_plddt = []
for index in df_3DAC["ID"]:
    if os.path.exists("3DAC_structure/%s/ranking_debug.json"%index):
        f_json = open("3DAC_structure/%s/ranking_debug.json"%index,"r")
        json_dict = json.load(f_json)
        try:
            plddt_1 = json_dict["plddts"]["model_1"]
            plddt_2 = json_dict["plddts"]["model_2"]
        except:
            plddt_1 = json_dict["plddts"]["model_1_ptm"]
            plddt_2 = json_dict["plddts"]["model_2_ptm"]
        template_list_plddt.append([index,plddt_1,plddt_2])
    else:
        template_list_plddt.append([index,None,None])
template_df_plddt = pd.DataFrame(template_list_plddt,columns=["ID","model_1_plddt","model_2_plddt"])
df_3DAC = df_3DAC.merge(template_df_plddt,how="left",on="ID")
df_3DAC.head()

,ID,name,sequence,sequence length,structure avail,model_1_plddt,model_2_plddt
0,3DAC_1,NaN,MAQKKGSSQKLLVWILVGFALGIVGGLILGKDNVIWVAWMGDVFIR...,409,True,85.773312,88.517354
1,3DAC_2,NaN,MSTMPSKVVGIIGGMGPEAGVDLVYKVIKLSPAKRDQEHIHVILDN...,232,True,96.532346,96.244699
2,3DAC_3,NaN,MAQVKEVRLYTYQWKAEPPIGVSVLIIHGLGEHAGRYKSLVKALND...,264,True,92.729368,92.923812
3,3DAC_4,NaN,MYSQFKKLVKDAIRKADIEKRLDEIITYRRELHQIPEVGLELPKTK...,394,True,94.767609,95.085076
4,3DAC_5,NaN,MNEMFVVQAKFIGKESQKKLEGKTVAIVGCGGLGSPLVQMAARSGI...,230,True,94.804814,95.198690


In [ ]:
# Calculate protein properties
disul_list = []
hbond_list = []
sbrge_list = []
surf__list = []
sasa__list = []
sstrc_list = []
for i in range(df_3DAC.shape[0]):
    if os.path.exists("3DAC_structure/%s/unrelaxed_model_1.pdb"%df_3DAC["ID"][i]):
        cmd.load("3DAC_structure/%s/unrelaxed_model_1.pdb"%df_3DAC["ID"][i],"%s_1"%df_3DAC["ID"][i])   # load pdb file

        cmd.distance("disul","name SG","name SG",cutoff=2.1)     # disulfide bond
        cmd.distance("hbond","all","all",mode=2)                 # hydrogen bond
        cmd.distance("saltbridge","(resn ASP and name OD2) or (resn GLU and name OD2)","(resn ARG and (name NH1 or name NH2)) or (resn LYS and name NZ) or (resn HIS and name NE2)",cutoff=4)  # salt bridge
        cmd.dss()
        ss_string = ""
        for a in cmd.get_model(df_3DAC["ID"][i] +" and n. ca").atom:
            ss_string = ss_string+a.ss 



        disul_list.append(len(get_raw_distances("disul")))       # disulfide bond
        hbond_list.append(len(get_raw_distances("hbond")))       # hydrogen bond
        sbrge_list.append(len(get_raw_distances("saltbridge")))  # salt bridge
        surf__list.append(cmd.get_area('all'))                   # protein surface
        cmd.set('dot_solvent',1)
        sasa__list.append(cmd.get_area('all'))                   # SASA
        sstrc_list.append(ss_string)                             # secondary structure

        cmd.reinitialize()
    else:
        disul_list.append(None)       # disulfide bond
        hbond_list.append(None)       # hydrogen bond
        sbrge_list.append(None)  # salt bridge
        surf__list.append(None)                   # protein surface
        sasa__list.append(None)                   # SASA
        sstrc_list.append(None)                             # secondary structure

    print("\r%d/%d"%(i+1,df_3DAC.shape[0]),end="")


10/10

In [ ]:
# Add calculated data to dataframe
df_3DAC["disulfide_bond"] = disul_list
df_3DAC["hydrogen_bond"] = hbond_list
df_3DAC["salt_bridge"] = sbrge_list
df_3DAC["surface"] = surf__list
df_3DAC["SASA"] = sasa__list
df_3DAC["secondary_structure"] = sstrc_list

df_3DAC.to_csv("3DAC_results.csv",index=False)
df_3DAC.head()

,ID,name,sequence,sequence length,structure avail,model_1_plddt,model_2_plddt,disulfide_bond,hydrogen_bond,salt_bridge,surface,SASA,secondary_structure
0,3DAC_1,NaN,MAQKKGSSQKLLVWILVGFALGIVGGLILGKDNVIWVAWMGDVFIR...,409,True,85.773312,88.517354,0,522,2,46165.542969,18625.396484,LLLLLHHHHHHHHHHHHHHHHHHHHHHHHLHHHHHHHHHHHHHHHH...
1,3DAC_2,NaN,MSTMPSKVVGIIGGMGPEAGVDLVYKVIKLSPAKRDQEHIHVILDN...,232,True,96.532346,96.244699,0,249,5,26818.630859,10773.984375,LLLLLLLLSSSSLLLLHHHHHHHHHHHHHHLLLLLHHHLLLSSSSL...
2,3DAC_3,NaN,MAQVKEVRLYTYQWKAEPPIGVSVLIIHGLGEHAGRYKSLVKALND...,264,True,92.729368,92.923812,0,304,12,30916.574219,12734.734375,LLLLLLLLLLSSSSLLLLLLLSSSSSSSLLLLLHHHHHHHHHHHHH...
3,3DAC_4,NaN,MYSQFKKLVKDAIRKADIEKRLDEIITYRRELHQIPEVGLELPKTK...,394,True,94.767609,95.085076,0,469,10,45688.320312,18097.576172,LHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHLLLLLLLLHHHH...
4,3DAC_5,NaN,MNEMFVVQAKFIGKESQKKLEGKTVAIVGCGGLGSPLVQMAARSGI...,230,True,94.804814,95.198690,0,258,5,26502.291016,10770.962891,LLHHHHHHHHHHLHHHHHHHHHSSSSSSLLLLHHHHHHHHHHHHLL...
